In [1]:
# import interface package and function for mapping objects between models
from CESIAPI.interface import MOD
from CESIAPI.MappingPassOver import CrossMapping

## model #1: planningmod

A power system investment model to determine the optimal installed capacity for each generator.

## model #2 operationalModel

A power system opteration model to determine the ouput of given generators to obtain the least overall operational cost.

## linking model #1 with model #2

1. run model #1, the capacities determined by planning model #1 will feed into operational model #2 as inputs

2. run model #2, valiate the performance of proposed generator mix. if any violaton of operational constraint occurs, then the degree of violaton will feed back to planning model as addional inputs (formulated using Benders decomposition) for its next run.

3. repeat step 1 and 2 until the two models convege.



In [2]:
# call planning model
planmod = MOD(lang='Julia')
planmod.newmodel(modelfile="planningmod.jl",jumpmod_name='rmp')

<PyCall.jlwrap Min cx + θ
Subject to
 cx - 86340 x_inv[1] - 21000 x_inv[2] - 35110 x_inv[3] - 15800 x_inv[4] - 191230 x_inv[5] == 0
 0 <= x_inv[1] <= 35000
 0 <= x_inv[2] <= 35000
 0 <= x_inv[3] <= 35000
 0 <= x_inv[4] <= 5000
 0 <= x_inv[5] <= 7000
 cx
 θ >= 0
>


In [3]:
# call operation model
opermod = MOD(lang='AMPL')
opermod.newmodel(modelfile="OperationalModel.run")

SUBPROBLEM


In [5]:
# declare the indexing sets of exchange items
# index_set_opermod = ['Gcoal', 'Gocgt', 'Gccgt', 'Gnucl', 'Gdies']
# index_set_planmod =  ['coal', 'ocgt', 'ccgt', 'diesel', 'nuclear']

mapping_opermod2planmod = {'Gcoal': 'coal',
                             'Gocgt': 'ocgt',
                             'Gccgt': 'ccgt',
                             'Gnucl': 'nuclear',
                             'Gdies': 'diesel'}

mapping_planmod2opermod = {'coal': 'Gcoal',
                            'ocgt': 'Gocgt',
                            'ccgt': 'Gccgt',
                            'diesel': 'Gdies',
                            'nuclear': 'Gnucl'}

In [4]:
ϵ = .001 # set the stop threshold value for iteration

In [ ]:
for i in list(range(30)):
    
    # solve planning model and get values of the exchange items
    planmod.solve()
    L = planmod.getObjectiveValue()
    x_fix = planmod.getValue('rmp[:x_inv][:]')

    # re-order the exchange items acording to the receiving model (operational model)
    # and pass over
    x_fix = CrossMapping(x_fix, mapping_planmod2opermod)
    opermod.getParam("pG_ub").setValues(x_fix)
    
    # solve operational mode and get values of the linking items
    opermod.solve()
    θ = opermod.getValue("oper")
#     θ = opermod.getObjectiveValue()
    λ = [c[1].dual() for c in opermod.getConstraint("lambda_pG")] #get constraint dual value into a list
    λ = CrossMapping(λ, mapping_opermod2planmod)

    # add addtional constraint (cut) into planning model
    # note: the bender algothrim is implemented in Julia, so involve some JuMP expression
    U = planmod.getValue('rmp[:cx]') + θ
    planmod.jumpworkspace.x_fix = x_fix # put this value back into Julia
    planmod.jumpworkspace.θ = θ
    planmod.jumpworkspace.λ = λ
    planmod.eval('@constraint(rmp,rmp[:θ] >= θ + sum(λ[i] * (rmp[:x_inv][i] - x_fix[i]) for i in 1:5))') # JuMP expression

    # check the gap, break if converge
    Δ = (U - L) / U * 100
    if Δ <= ϵ:
        break

In [7]:
# print the results
print(" ")
print("---------------------------------------------------------------------")
print("Investment Decisions")
print("coal    -> {} GW".format(round(x_fix[0] / 1000)))
print("ocgt    -> {} GW".format(round(x_fix[1] / 1000)))
print("ccgt    -> {} GW".format(round(x_fix[2] / 1000)))
print("diesel  -> {} GW".format(round(x_fix[3] / 1000)))
print("nuclear -> {} GW".format(round(x_fix[4] / 1000)))
print(" ")

print("Cost for investment -> {} m£".format(round(planmod.getValue('rmp[:cx]') / 1e6)))
print("Cost for operation  -> {} m£".format(round(θ / 1e6)))

 
---------------------------------------------------------------------
Investment Decisions
coal    -> 0.0 GW
ocgt    -> 35.0 GW
ccgt    -> 12.0 GW
diesel  -> 5.0 GW
nuclear -> 0.0 GW
 
Cost for investment -> 1220 m£
Cost for operation  -> 6176 m£
